In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv')

In [ ]:
data.head()

In [ ]:
pd.set_option('display.max_rows' , None)
data.isnull().sum()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
# import missingno as msno
# plt.figure(figsize = (20,10))
# msno.bar(data)
# plt.show()

In [ ]:
missing_value_counts =pd.DataFrame(data.isnull().sum())
missing_value_counts.reset_index()
missing_value_counts.plot()

In [ ]:
plt.figure(figsize = (20,10))
sns.heatmap(data.isnull() , cbar = False)
plt.show()

In [ ]:
#percent of missing value
a = np.product(data.shape)
b = data.isnull().sum().sum()
(b/a) * 100

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import seaborn as sns
primary_bgcolor = "#f4f0ea"
primary_blue = "#496595"
primary_blue2 = "#85a1c1"
primary_blue3 = "#3f4d63"
primary_grey = "#c6ccd8"
primary_black = "#202022"
corr = data.corr().abs()
mask = np.triu(np.ones_like(corr, dtype=bool))

fig, ax = plt.subplots(figsize=(90, 50), facecolor=primary_bgcolor)
# ax.text(-1.1, 0.16, 'Correlation between the Continuous Features', fontsize=10, fontweight='bold', fontfamily='serif')
ax.text(-1.1, 0.3, 'There is no features that pass more than 0.32 correlation within each other', fontsize=13, fontweight='light', fontfamily='serif')


# plot heatmap
res=sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='coolwarm',annot_kws={"size": 25},
                            cbar_kws={"shrink": .2}, vmin=0, vmax=1)
res.set_xticklabels(res.get_xmajorticklabels(), fontsize = 28)
res.set_yticklabels(res.get_ymajorticklabels(), fontsize = 28)
# yticks
plt.yticks(rotation=0)
plt.show()

In [ ]:
features = data.columns
F = [[] , [] , [] , [] , []]
for feature in features:
    for i in [1,2,3,4]:
        if(feature.split('_')[1] == str(i)):
            F[i].append(feature)

df = [[] , [] , [] , [] , []]
fig , axes = plt.subplots(nrows = 4 , ncols = 1 , figsize = (18,30))
for i in [1,2,3,4]:
    df[i] = data[F[i]]
    corr = df[i].corr().abs()
    sns.heatmap(corr , annot = True , ax = axes[i-1])

In [ ]:
%%time
import xgboost
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
for i in [1,3,4]:
    imptr = IterativeImputer(
    estimator=xgboost.XGBRegressor(
        n_estimators=100,
        random_state=42,
        tree_method='gpu_hist',
    ),
    missing_values=np.nan,
    max_iter=10,
    initial_strategy='mean',
    imputation_order='ascending',
    verbose=2,
    random_state=42
    )
    df[i][:] = imptr.fit_transform(df[i])

In [ ]:
%%time
d = pd.concat([df[1] , df[2] , df[3] , df[4]] , axis = 1) 

In [ ]:
sub = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv' , index_col='row-col')

In [ ]:
from tqdm import tqdm
for i in tqdm(sub.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    sub.loc[i, 'value'] = d.loc[row, col]

sub.to_csv('submission.csv')